# MalDroid Data Ingest
This notebook handles the retrieval of MalDroid malware sample analysis (sample_for_analysis.apk.json) files from the MalDroid repo (http://205.174.165.80/CICDataset/MalDroid-2020/Dataset/Capturing_logs/). 
## Process
1. Take the URIs of the .tar.gz files for each sample from MalDroid_ref_raw.csv, extract them to .tar and then to a directory *malware class*/*hash*
2. Check to ensure the sample_for_analysis.apk.json file does not throw an error when opening. If yes, directory will be deleted and script will move to next sample
3. Move the sample_for_analysis.apk.json file from the *malware class*/*hash*/sample_for_analysis.apk subdirectory to under *malware class*/*hash*/ and delete the subdirectory

In [ ]:
import pandas as pd 
import tarfile
import os
import urllib.request
import json
import shutil

In [ ]:
with open('MalDroid_ref_raw.csv') as ref_raw_path:
    ref_raw = pd.read_csv(ref_raw_path)
ref_raw = ref_raw[['hash', 'URI', 'malware_class']]
ref_raw = ref_raw.drop([13076])
# Drops final row; does not contain a sample due to parsing issues

In [ ]:
issue_hashes = []

for sample in ref_raw.itertuples(index=False):
    print(sample.URI)
    if '.tar.gz' in sample.URI:
        base_path = sample.malware_class.lower() + '/' + sample.hash
        temp_file = urllib.request.urlretrieve(sample.URI, filename=None)[0]
        file = tarfile.open(temp_file)
        file.extractall(base_path)
        file.close()
        with open(base_path + '/sample_for_analysis.apk/sample_for_analysis.apk.json') as default_path:
            try:
                json.load(default_path)
            except:
                issue_hashes.append(sample.hash)
                continue
        shutil.move(base_path + '/sample_for_analysis.apk/sample_for_analysis.apk.json', base_path + '/sample_for_analysis.apk.json')
        shutil.rmtree(base_path + '/sample_for_analysis.apk')